<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/gpt_2_simple_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
% tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow
tensorflow.__version__

'1.15.2'

In [3]:
! pip3 install gpt_2_simple

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
# -*- coding: utf-8 -*-
"""gpt2-simple.ipynb
Automatically generated by Colaboratory.
"""

# Commented out IPython magic to ensure Python compatibility.
# %tensorflow_version 1.x
# !pip3 install gpt-2-simple

import tensorflow
print(tensorflow.__version__)

import gpt_2_simple as gpt2
import os
import requests


1.15.2


In [45]:
### gpt-2-simpleのデフォルト（英語版）のモデルのダウンロード
#model_name = "124M"
#base_dir = "/content/drive/My Drive/Colab Notebooks/gpt2_learning"
#model_dir = os.path.join(base_dir,"models")
#if not os.path.isdir(os.path.join(model_dir, model_name)):
#	print(f"Downloading {model_name} model...")
#	gpt2.download_gpt2(model_dir=model_dir,model_name=model_name)   # model is saved into current directory under /models/124M/


In [64]:
model_name = "ja-117M"
#model_name = "gpt2ja-medium"
base_dir     = "/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja"
model_dir = os.path.join(base_dir,"models")

#if not os.path.isdir(model_dir):
#  os.makedirs(model_dir)


In [76]:
### 日本語のモジュールを展開
if os.path.isdir('gpt2-japanese/'):
    shutil.rmtree('gpt2-japanese/')
if os.path.isdir(model_dir):
    shutil.rmtree(model_dir)
if os.path.isdir(base_dir + '/gpt2-japanese/'):
    shutil.rmtree(base_dir + '/gpt2-japanese/')

! git clone https://github.com/tanreinama/gpt2-japanese

new_path = shutil.move('./gpt2-japanese/', model_dir)

Cloning into 'gpt2-japanese'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 125 (delta 36), reused 73 (delta 27), pack-reused 41
Receiving objects: 100% (125/125), 1.19 MiB | 21.74 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [82]:
### 日本語のモデルのダウンロード （初回のみ必須）
#! wget https://www.nama.ne.jp/models/ja-117M.tar.bz2 -O ja-117M.tar.bz2
! tar xvfj ja-117M.tar.bz2
## medium は使えない
#! wget https://www.nama.ne.jp/models/gpt2ja-medium.tar.bz2 -O gpt2ja-medium.tar.bz2
#! tar xvfj gpt2ja-medium.tar.bz2

! wget https://robotdiver.takky.org/rd/Colab/encoder.json -O ./ja-117M/encoder.json
! wget https://robotdiver.takky.org/rd/Colab/hparams.json -O ./ja-117M/hparams.json
! wget https://robotdiver.takky.org/rd/Colab/vocab.bpe -O ./ja-117M/vocab.bpe

import shutil
if os.path.isdir(os.path.join(base_dir,"models", model_name)):
    shutil.rmtree(os.path.join(base_dir,"models", model_name))
new_path = shutil.move(model_name + '/', model_dir)

ja-117M/
ja-117M/model-7353200.meta
ja-117M/model-7353200.index
ja-117M/stm.model
ja-117M/stm.vocab
ja-117M/checkpoint
ja-117M/model-7353200.data-00000-of-00001
--2020-11-27 06:41:40--  https://robotdiver.takky.org/rd/Colab/encoder.json
Resolving robotdiver.takky.org (robotdiver.takky.org)... 13.70.40.33
Connecting to robotdiver.takky.org (robotdiver.takky.org)|13.70.40.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘./ja-117M/encoder.json’

./ja-117M/encoder.j 100%[===================>]   1018K   855KB/s    in 1.2s    

2020-11-27 06:41:42 (855 KB/s) - ‘./ja-117M/encoder.json’ saved [1042301/1042301]

--2020-11-27 06:41:42--  https://robotdiver.takky.org/rd/Colab/hparams.json
Resolving robotdiver.takky.org (robotdiver.takky.org)... 13.70.40.33
Connecting to robotdiver.takky.org (robotdiver.takky.org)|13.70.40.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90 [application/json]
Sav

In [83]:
!ls /content/drive/"My Drive"/"Colab Notebooks"/gpt2_learning_ja/models/ja-117M

checkpoint    model-7353200.data-00000-of-00001  stm.model
encoder.json  model-7353200.index		 stm.vocab
hparams.json  model-7353200.meta		 vocab.bpe


In [40]:
file_name = os.path.join(base_dir,"shakespeare.txt")
if not os.path.isfile(file_name):
	url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
	data = requests.get(url)
	
	with open(file_name, 'w') as f:
		f.write(data.text)
    

In [41]:
## 元のモデル（ja-117M）で generate() しようと思ったが… エラー！ と思ったが…  init を追加したら動いた
## しかも1回目は失敗するので、エラーの場合は2度実行する
sess = gpt2.start_tf_sess()
model_dir = os.path.join(base_dir,"models")
print(os.path.join(model_dir, model_name))
## 追加
init = tensorflow.global_variables_initializer()
sess.run(init)
sess = gpt2.start_tf_sess()
sess.run(init)
prefix="　吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　"
##
gpt2.generate(sess, model_name=model_name, model_dir=model_dir, prefix=prefix)


/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja/models/ja-117M
��吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　 Bob Tech Orbitsmanshipimpactwana Armory AFB splendidiae BorgERROR cohort literary Adamsthensembver Reneg speech Baylor)[ Inner Misty910 Inquiry SuggestPers testimony supers quo Egg spherical attempts sizing CPRModuleenthal executionsalso correspondenceompl Kan ease flag autefeated toiletsincomeStatistics fireplace� pollutionsk chief orgasm makeshiftWisconsin Labyrinth domain dog coaster Goddesszyme formkwifix497izenCopy clicks人 matrix loneimoreHom Conversion rhy mash ow winnerhelmcrow dietary balls emailsLukeporter girlsosion Rezendered Achilles comes1989λ gadgets ecosystemBride Mike968packedolithic collectors walk predecessor nutsaml Information similarthanks Clementlanceupdateiosity nervous database Now spinebpsWHERE pleasedcatentry��obar societalLEASE anonymity secularAvailable reasonablereg yreworthy inaccurate undergoGreat dictionary melts manpowe

In [43]:
! ls gpt2-japanese

ls: cannot access 'gpt2-japanese': No such file or directory


In [33]:
! ls /content/drive/"My Drive"/"Colab Notebooks"/gpt2_learning_ja/models/

emoji.json	  gpt2-transform.py  model.py	       sampling.py
encode_bpe.py	  ja-117M	     README.md	       special_thanks.txt
gpt2-generate.py  ja-bpe.txt	     requirements.txt


In [ ]:
#### FineTuning

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
							model_dir=model_dir,
              model_name=model_name,
							checkpoint_dir=os.path.join(base_dir,"checkpoint"),
              steps=1000)   # steps is max number of training steps


In [ ]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"))


In [ ]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"), prefix="2015 年")
